In [19]:
import pandas as pd
import numpy as np
import re

In [20]:
pd.options.display.max_columns = None

## Cargamos los datos

In [21]:
df_unidos = pd.read_csv('../datos/datos_unidos.csv', index_col = 0)
df_unidos.head(2)

,age,gender,Q3,Q5,Q6,Q8,Q15,Q22,Q41,Q7,Q9,Q24,Q32,Q34,Q10_Part_1,Q10_Part_2,Q10_Part_3,Q10_Part_4,Q10_Part_5,Q10_Part_6,Q10_Part_7,Q10_Part_8,Q10_Part_9,Q10_Part_10,Q10_Part_11,Q10_Part_12,Q10_Part_13,Q10_Part_14,Q10_Part_15,Q10_Part_16,Q10_OTHER,Q39_Part_1,Q39_Part_2,Q39_Part_3,Q39_Part_4,Q39_Part_5,Q39_Part_6,Q39_Part_7,Q39_Part_8,Q39_Part_9,Q39_OTHER
0,50-54,Man,India,Other,5-10 years,Python,5-10 years,3-4,Local development environments (RStudio; Jupyt...,Python; R,Vim / Emacs,None of these activities are an important par...,PostgreSQL,None,NaN,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GitHub,NaN,Kaggle,NaN,NaN,NaN,NaN
1,50-54,Man,Indonesia,Program/Project Manager,20+ years,Python,< 1 year,1-2,Advanced statistical software (SPSS; SAS; etc.),SQL; C; C++; Java,Notepad++;Jupyter Notebook,Build and/or run the data infrastructure that...,NaN,NaN,Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_da_ba = pd.read_csv('../datos/datos_da_ba.csv', index_col=0)
df_da_ba.head(2)

,age,gender,Q3,Q5,Q6,Q8,Q15,Q22,Q41,Q7,Q9,Q24,Q32,Q34,Q10_Part_1,Q10_Part_2,Q10_Part_3,Q10_Part_4,Q10_Part_5,Q10_Part_6,Q10_Part_7,Q10_Part_8,Q10_Part_9,Q10_Part_10,Q10_Part_11,Q10_Part_12,Q10_Part_13,Q10_Part_14,Q10_Part_15,Q10_Part_16,Q10_OTHER,Q39_Part_1,Q39_Part_2,Q39_Part_3,Q39_Part_4,Q39_Part_5,Q39_Part_6,Q39_Part_7,Q39_Part_8,Q39_Part_9,Q39_OTHER
16,50-54,Man,Belgium,Data Analyst,20+ years,Python,1-2 years,5-9,Local development environments (RStudio; Jupyt...,Python; SQL,Jupyter (JupyterLab; Jupyter Notebooks; etc) ;...,Analyze and understand data to influence produ...,NaN,NaN,Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,22-24,Nonbinary,United States of America,Data Analyst,< 1 years,R,I do not use machine learning methods,3-4,NaN,R,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_da_ba.shape

(3269, 41)

## Extraemos los valores únicos de cada columna

In [24]:
#esta función crea un dicionario con el número de la columna como keys y una lista de los valores únicos de la columna como values
def dict_respuestas(dataframe):
    
    dic_respuestas = {}
    
    for col in dataframe.columns:
        lista_nueva = []
        #ignoramos la columna index
        if col == "index":
            pass
        #comprobamos si los valores únicos son más de 20 - si es una pregunta multirespuesta 
        elif len(dataframe[col].unique()) > 22:
            #crea una lista de los valores únicos
                lista_unicos = list(dataframe[col].unique())
                for sublist in lista_unicos:
                    #cada lista dentro de los valores únicos se divide por las comas 
                    try:
                        lista_nueva.extend(sublist.split(","))
                    except:
                        pass
                    #lista de todas las posibles respuestas individuales
                    #la convertimos en set para eliminar los duplicados 
                    set_unicos = set(lista_nueva)
                    #lo volvemos a convertir en lista para poder usarla cómodamente
                    lista_nueva = list(set_unicos)
        else:
            #si no es multirespuesta podemos usar el unique para ver los valores únicos 
            lista_nueva = list(dataframe[col].unique())
        #list comprehension para quitar los nulos    
        lista_sin_nan = [item for item in lista_nueva if not (pd.isnull(item)) == True]
        #metemos la lista sin nulos en un diccionario donde el key es el número de la columna y los values con la lista de valores únicos lista_sin_nan
        dic_respuestas[col] = lista_sin_nan
        
        print("-------------------")
        print (f"{col} : {dic_respuestas[col]}")

    return dic_respuestas


In [25]:
diccionario_respuestas = dict_respuestas(df_da_ba)

-------------------
age : ['50-54', '22-24', '30-34', '40-44', '70+', '45-49', '18-21', '55-59', '25-29', '35-39', '60-69']
-------------------
gender : ['Man', 'Nonbinary', 'Woman', 'Prefer not to say', 'Prefer to self-describe']
-------------------
Q3 : ['Egypt', 'Turkey', 'Colombia', 'Nepal', 'Poland', 'Viet Nam', 'Nigeria', 'Netherlands', 'Belarus', 'Argentina', 'Norway', 'Pakistan', 'Canada', 'Italy', 'Bangladesh', 'Taiwan', 'Greece', 'India', 'Ethiopia', 'Saudi Arabia', 'Tunisia', 'Portugal', 'Denmark', 'United Kingdom of Great Britain and Northern Ireland', 'Ukraine', 'United States of America', 'Switzerland', 'Uganda', 'Ghana', 'Kenya', 'Kazakhstan', 'Iraq', 'Other', 'I do not wish to disclose my location', 'Mexico', 'Czech Republic', 'Ecuador', 'Sri Lanka', 'Philippines', 'Romania', 'South Africa', 'Japan', 'Brazil', 'Indonesia', 'Iran; Islamic Republic of...', 'Thailand', 'Malaysia', 'Sweden', 'Singapore', 'Germany', 'Spain', 'Chile', 'Australia', 'United Arab Emirates', 'Fra

## Contamos los valores y calculamos el %

In [26]:
#devuelve el valor de cada valor único y su porcentaje
#el diccionario abajo es el diccionario de respuestas que acabamos de hacer 
def count_respuestas(dataframe, **diccionario):
    for key in diccionario:
        #usamos el diccionario, sacamos los valores únicos de cada pregunta para encontrar la columna del dataframe, 
        # y el sum nos dice cuántas veces aparece      
        if key == "index":
            pass
        elif len(dataframe[key].unique()) > 22:
            print("----------------")
            print(f"PREGUNTA MULTIRESPUESTA {key}")  
            for respuesta in diccionario[key]:
                try:
                    total = dataframe[key].str.contains(respuesta, regex = True).sum()
                    print(f"{respuesta}: total: {total}, {round(total/dataframe.shape[0]*100, 1)}%")
                except:
                    print(f"no se puede para {respuesta}")
        else:
            print("------------")
            print(f"PREGUNTA {key} (%)")
            print(f"{round((dataframe[key].value_counts()[:-1]/dataframe.shape[0]*100),2)}")


In [27]:
count_respuestas(df_da_ba, **diccionario_respuestas)

------------
PREGUNTA age (%)
25-29    24.11
30-34    18.05
22-24    16.46
35-39    12.70
40-44     8.26
18-21     7.22
45-49     5.41
50-54     3.70
60-69     1.93
55-59     1.80
Name: age, dtype: float64
------------
PREGUNTA gender (%)
Man                  77.64
Woman                20.80
Prefer not to say     1.16
Nonbinary             0.24
Name: gender, dtype: float64
----------------
PREGUNTA MULTIRESPUESTA Q3
Egypt: total: 101, 3.1%
Turkey: total: 36, 1.1%
Colombia: total: 41, 1.3%
Nepal: total: 5, 0.2%
Poland: total: 33, 1.0%
Viet Nam: total: 32, 1.0%
Nigeria: total: 132, 4.0%
Netherlands: total: 20, 0.6%
Belarus: total: 6, 0.2%
Argentina: total: 32, 1.0%
Norway: total: 3, 0.1%
Pakistan: total: 50, 1.5%
Canada: total: 49, 1.5%
Italy: total: 31, 0.9%
Bangladesh: total: 31, 0.9%
Taiwan: total: 41, 1.3%
Greece: total: 15, 0.5%
India: total: 795, 24.3%
Ethiopia: total: 2, 0.1%
Saudi Arabia: total: 16, 0.5%
Tunisia: total: 5, 0.2%
Portugal: total: 15, 0.5%
Denmark: total: 3, 0.1%
Un

/tmp/ipykernel_16100/1125823183.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  total = dataframe[key].str.contains(respuesta, regex = True).sum()


Python; SQL; Bash; Other: total: 4, 0.1%
no se puede para Python; C++; Java; MATLAB
no se puede para Python; SQL; C; C++; Java; Javascript; Bash; MATLAB
Python; Java; Bash: total: 2, 0.1%
no se puede para  C++
Python; Bash: total: 4, 0.1%
Python; R; SQL; Java; Javascript; MATLAB: total: 1, 0.0%
Python; Javascript; Bash: total: 1, 0.0%
no se puede para Python; R; SQL; C++; Javascript; MATLAB
no se puede para Python; R; C; C++; MATLAB
no se puede para Python; C; C++; Bash; MATLAB
no se puede para Python; C++; MATLAB
Python; SQL; MATLAB: total: 32, 1.0%
Python; R; SQL; Javascript; MATLAB: total: 1, 0.0%
 R; SQL; Bash: total: 21, 0.6%
no se puede para Python; SQL; C; C++
Python; R; SQL; C; Java; Javascript: total: 4, 0.1%
no se puede para  SQL; C; C++; Java; Javascript
Python; R; SQL; Javascript; Bash; Other: total: 1, 0.0%
 R; Java; Swift: total: 1, 0.0%
no se puede para Python; R; C; C++; Java; MATLAB
Python; SQL; Julia: total: 1, 0.0%
no se puede para Python; SQL; C; C++; Java
no se pue

/tmp/ipykernel_16100/1125823183.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  total = dataframe[key].str.contains(respuesta, regex = True).sum()


PyCharm ; Sublime Text: total: 54, 1.7%
RStudio ;Visual Studio ;Jupyter Notebook: total: 10, 0.3%
Jupyter (JupyterLab; Jupyter Notebooks; etc) ;MATLAB ;Jupyter Notebook: total: 0, 0.0%
no se puede para Jupyter (JupyterLab; Jupyter Notebooks; etc) ; Notepad++;MATLAB ;Jupyter Notebook; Other
RStudio ;PyCharm : total: 110, 3.4%
RStudio ;PyCharm ;Jupyter Notebook: total: 43, 1.3%
no se puede para Visual Studio Code (VSCode) ;PyCharm ; Spyder; Notepad++; Sublime Text;Jupyter Notebook
no se puede para Visual Studio ;PyCharm ; Notepad++;Jupyter Notebook
no se puede para Visual Studio ;Visual Studio Code (VSCode) ; Notepad++; Other
no se puede para Visual Studio Code (VSCode) ;PyCharm ; Notepad++; Sublime Text;Jupyter Notebook; Other
no se puede para Jupyter (JupyterLab; Jupyter Notebooks; etc) ;Visual Studio Code (VSCode) ; Notepad++; Sublime Text; Vim / Emacs;Jupyter Notebook
no se puede para  Notepad++;MATLAB ;Jupyter Notebook
no se puede para RStudio ;Visual Studio ;Visual Studio Code (VSC

## Categorización por años de experiencia

In [29]:
df_unidos.head(2)

,age,gender,Q3,Q5,Q6,Q8,Q15,Q22,Q41,Q7,Q9,Q24,Q32,Q34,Q10_Part_1,Q10_Part_2,Q10_Part_3,Q10_Part_4,Q10_Part_5,Q10_Part_6,Q10_Part_7,Q10_Part_8,Q10_Part_9,Q10_Part_10,Q10_Part_11,Q10_Part_12,Q10_Part_13,Q10_Part_14,Q10_Part_15,Q10_Part_16,Q10_OTHER,Q39_Part_1,Q39_Part_2,Q39_Part_3,Q39_Part_4,Q39_Part_5,Q39_Part_6,Q39_Part_7,Q39_Part_8,Q39_Part_9,Q39_OTHER
0,50-54,Man,India,Other,5-10 years,Python,5-10 years,3-4,Local development environments (RStudio; Jupyt...,Python; R,Vim / Emacs,None of these activities are an important par...,PostgreSQL,None,NaN,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GitHub,NaN,Kaggle,NaN,NaN,NaN,NaN
1,50-54,Man,Indonesia,Program/Project Manager,20+ years,Python,< 1 year,1-2,Advanced statistical software (SPSS; SAS; etc.),SQL; C; C++; Java,Notepad++;Jupyter Notebook,Build and/or run the data infrastructure that...,NaN,NaN,Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_unidos['Q6'].unique()

array(['5-10 years', '20+ years', '1-3 years', '< 1 years', '3-5 years',
       '10-20 years', 'I have never written code'], dtype=object)

6, df["number_of_policies"] < 6]

creamos la lista de los valores que querremos crear cuando se cumpla cada condición
opciones = ["supreme", "good"]

aplicamos el método a nuestro DataFrame y creamos una nueva columna llamada "new_select"
df["new_select"] = np.select(condiciones, opciones)

In [34]:
condiciones = [(df_unidos['Q6'] == 'I have never written code'), 
                ((df_unidos['Q6'] == '< 1 years')|(df_unidos['Q6'] == '1-3 years')), 
                ((df_unidos['Q6'] == '3-5 years')| (df_unidos['Q6'] == '5-10 years')|(df_unidos['Q6'] == '10-20 years')|(df_unidos['Q6'] == '20+ years'))]
categorias = ['Sin experiencia', 'Junior', 'Senior'] 

In [35]:
df_unidos['experiencia programacion'] = np.select(condiciones, categorias)

In [45]:
df_da_ba = df_unidos[(df_unidos ["Q5"] == "Business Analyst") | (df_unidos ["Q5"] == "Data Analyst")]

In [46]:
# df con filtrado por analista de datos y business analyst con columna categórica por experiencia creada
df_da_ba.head(1)

,age,gender,Q3,Q5,Q6,Q8,Q15,Q22,Q41,Q7,Q9,Q24,Q32,Q34,Q10_Part_1,Q10_Part_2,Q10_Part_3,Q10_Part_4,Q10_Part_5,Q10_Part_6,Q10_Part_7,Q10_Part_8,Q10_Part_9,Q10_Part_10,Q10_Part_11,Q10_Part_12,Q10_Part_13,Q10_Part_14,Q10_Part_15,Q10_Part_16,Q10_OTHER,Q39_Part_1,Q39_Part_2,Q39_Part_3,Q39_Part_4,Q39_Part_5,Q39_Part_6,Q39_Part_7,Q39_Part_8,Q39_Part_9,Q39_OTHER,experiencia programacion
16,50-54,Man,Belgium,Data Analyst,20+ years,Python,1-2 years,5-9,Local development environments (RStudio; Jupyt...,Python; SQL,Jupyter (JupyterLab; Jupyter Notebooks; etc) ;...,Analyze and understand data to influence produ...,NaN,NaN,Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Senior


In [52]:
df_group_genero = df_da_ba.groupby(['gender', 'experiencia programacion'])['age'].count().reset_index()
df_group_genero.head(2)

,gender,experiencia programacion,age
0,Man,Junior,1443
1,Man,Senior,891


In [56]:
df_crosstab_gen_exp = pd.crosstab(df_da_ba['gender'], df_da_ba['experiencia programacion'], normalize = True) * 100
df_crosstab_gen_exp

experiencia programacion,Junior,Senior,Sin experiencia
gender,,,
Man,44.141939,27.256042,6.240441
Nonbinary,0.122362,0.091771,0.030590
Prefer not to say,0.550627,0.520037,0.091771
Prefer to self-describe,0.061181,0.061181,0.030590
Woman,12.664423,6.118079,2.018966


In [61]:
# Sacamos tabla de contigencia cruzando género y la pregunta q8, donde se recoge la información de recomendación sobre herramientas de programación para iniciarse en análisis de datos

df_crosstab_gen_herr = pd.crosstab(df_da_ba['gender'], df_da_ba['Q8'], normalize = True, margins = True) * 100
df_crosstab_gen_herr.T

gender,Man,Nonbinary,Prefer not to say,Prefer to self-describe,Woman,All
Q8,,,,,,
Bash,0.133645,0.000000,0.033411,0.000000,0.033411,0.200468
C,0.601403,0.000000,0.000000,0.000000,0.233879,0.835282
C++,1.002339,0.000000,0.033411,0.000000,0.267290,1.303040
Java,0.701637,0.000000,0.000000,0.000000,0.300702,1.002339
Javascript,0.334113,0.000000,0.066823,0.000000,0.033411,0.434347
Julia,0.367524,0.033411,0.000000,0.000000,0.066823,0.467758
MATLAB,0.434347,0.000000,0.000000,0.000000,0.233879,0.668226
None,0.367524,0.000000,0.033411,0.000000,0.066823,0.467758
Other,0.735048,0.000000,0.000000,0.000000,0.066823,0.801871
